In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from cytoolz import concat
import re
import os

def anger_(l):
    x = [w for w in re.findall(r'\bliar\b|\bliars\b|\blying\b|\blies\b|\bhypocrite\b|\bhypocrites\b|\bhypocrisy\b|\bhypocritical\b|\basshole\b|\bassholes\b|\bbullshit\b|\bdisgrace\b|\bdisgraces\b|\bdisgraced\b|\bdisgraceful\b|\bstfu\b|\bdisgusting\b|\bdisgusted\b|\bdisgusts\b|\bscum\b|\binfuriate\b|\binfuriates\b|\binfuriating',l)]
    a = [w for w in re.findall(r'\bfuck you\b|\bthe fuck up\b|\bpiece of shit\b|\bgo fuck yourself\b',l)]
    b = [w for w in re.findall(r'\b(piss)\b.*off',l)]
    c = [w for w in re.findall(r'\boff\b.*\b(piss)\b',l)]
    d = [w for w in re.findall(r'\b(fuck)\b.*\boff\b',l)]
    e = [w for w in re.findall(r'\boff\b.*\b(fuck)\b',l)]
    z = []
    lists = [x,a,b,c,d,e,]
    for q in lists:
        for s in q:
            z.append(s)
    return z

#Count occurrences of anger words
def angertopic(filepath):
    df = pd.read_excel(filepath)
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False]
    df = df[df['TEXT'].str.contains('HDMA')==False]
    df = df[df['LANGUAGE']=='en'].copy()
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df = df[df['TEXT'].str.contains('FALSE')==False].copy()
    df = df[df['TEXT'].str.contains('TRUE')==False].copy()
    df['anger words'] = df['TEXT'].str.lower().apply(anger_)
    words = list(concat(df['anger words']))
    wordcounts = Counter(words)
    if wordcounts == Counter():
        dic = {'Anger Words': 0, 'Tweet Count': len(df)}
        counts = pd.DataFrame.from_dict(dic,orient='index')
        counts.columns = ['Count']
    else:
        keywords = {'liar':0, 'liars':0,'lying':0, 'lies':0, 
                'hypocrite':0,'hypocrites':0, 'hypocrisy':0, 'hypocritical':0,
                'asshole':0, 'assholes':0, 'bullshit':0, 
                'fuck':0, 'fuck you':0, 
                'disgrace':0, 'disgraces':0, 'disgraced':0, 'disgraceful':0, 
                'piece of shit':0, 'the fuck up':0, 
                'piss':0, 'stfu':0,
                'disgusting':0, 'disgusted':0, 'disgusts':0, 
                'go fuck yourself':0, 'scum':0, 
                'infuriate':0, 'infuriates':0, 'infuriating':0, 'infuriated':0}
        kw = pd.DataFrame.from_dict(keywords, orient='index')
        wc = pd.DataFrame.from_dict(wordcounts, orient='index')
        counts = kw.join(wc, how='left', lsuffix = 'kw', rsuffix = 'wc').fillna(0)
        counts['Counts'] = counts['0kw'] + counts['0wc']
        del counts['0kw']
        del counts['0wc']
        if 'fuck' in counts.index:
            counts.index.set_value(counts.index, 'fuck', 'fuck off')
        if 'piss' in counts.index:
            counts.index.set_value(counts.index, 'piss', 'piss(ed) off')
        counts.loc['~Total'] = counts.sum()
        counts.loc['~Tweet Count'] = len(df)
        counts['Percent'] = (counts['Counts']/len(df))*100
    name = os.path.basename(filepath).split('.')[0] + ' anger.csv'
    counts.sort_index(axis=0, ascending=True).to_csv(os.path.join(os.path.dirname(filepath),name))

def angertopics(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            angertopic(os.path.join(directory, filename))
            continue
        else:
            continue

In [3]:
angertopic('INSERT FILEPATH')

In [ ]:
angertopics('DIRECTORY')